In [1]:
# pip install accelerate
import torch
torch.cuda.empty_cache()
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import warnings
warnings.filterwarnings('ignore')

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", quantization_config=quantization_config)

stack = """
   0x0000003ff7ecdcfe <+0>:	add	sp,sp,-32
   0x0000003ff7ecdd00 <+2>:	sd	s0,16(sp)
   0x0000003ff7ecdd02 <+4>:	sd	s1,8(sp)
   0x0000003ff7ecdd04 <+6>:	sd	s2,0(sp)
   0x0000003ff7ecdd06 <+8>:	sd	ra,24(sp)
   0x0000003ff7ecdd08 <+10>:	add	s0,sp,32
   0x0000003ff7ecdd0a <+12>:	mv	s2,a0
   0x0000003ff7ecdd0c <+14>:	mv	a0,a1
   0x0000003ff7ecdd0e <+16>:	mv	s1,a1
   0x0000003ff7ecdd10 <+18>:	jal	0x3ff7ece0da <strlen>
   0x0000003ff7ecdd14 <+22>:	ld	s0,16(sp)
   0x0000003ff7ecdd16 <+24>:	ld	ra,24(sp)
   0x0000003ff7ecdd18 <+26>:	add	a2,a0,1
   0x0000003ff7ecdd1c <+30>:	mv	a1,s1
   0x0000003ff7ecdd1e <+32>:	mv	a0,s2
   0x0000003ff7ecdd20 <+34>:	ld	s1,8(sp)
   0x0000003ff7ecdd22 <+36>:	ld	s2,0(sp)
   0x0000003ff7ecdd24 <+38>:	add	sp,sp,32
   0x0000003ff7ecdd26 <+40>:	j	0x3ff7ecbcc4 <memcpy>
"""

f = open("learn.txt", "r")
learn = f.read()
f.close

print("In-text:", learn)
input_ids = tokenizer(learn, return_tensors="pt").to("cuda")



/home/rebus/Repo/RISCV-Attacks/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards:   0%|          | 0/4 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=150)
print(tokenizer.decode(outputs[0]))

question = "Fill the buffer and run out of the function. The output must be python3 -c 'print(\"A\"*x + exploit)' where the exploit is the shellcode. What is the value of x?"


print("In-text:", stack, question)
input_ids = tokenizer(stack + question, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=150)
print(tokenizer.decode(outputs[0]))


/home/rebus/Project/.env/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
2024-03-07 19:16:30.742009: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 19:16:30.757256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 19:16:30.757269: E external/local_xla/xla/stream_executor/cuda/cuda

<bos>Buffer Overflow on RISCV64 (sifive u74-mc)
In this scenario it will be used the ISA rv64imafdc. this time the vulnerable code will be:

void not_called() {
printf("Enjoy your shell!\n");
system("/bin/bash");
}
int test_empty() {
printf("Empty function\n");
return 1;
}
void vulnerable_function(char* string) {
char buffer[100];
test_empty();
strcpy(buffer, string);
}
int main(int argc, char** argv) {
vulnerable_function(argv[1]);
return 0;
}

The attacker will exploit the following vulnerability:

- The function `vulnerable_function` takes a pointer to a string `string` as input.
- The function allocates a local buffer `buffer` of 100 bytes.
- The function calls the function `strcpy` to copy the contents of the input string `string` into the buffer `buffer`.
- If the input string `string` is longer than 100 bytes, it will cause a buffer overflow, and the program will crash.

The attacker can exploit this vulnerability by passing a string that is longer than 100 bytes to the function